# Adaptive Contrast Enhancement of Grayscale Images Using Local Statistics and Bilateral Filtering

### Jave Atanacio Bacsain BSCS-2A (As a requirement for the course of Digital Image Processing - CSAC 225)
# FINAL PROJECT

- Note that details are already indicated in the document `FP_Bacsain_BSCS2A.pdf`
- Ran on Python 3.12.0

# All dataset images processed with PSNR and SSIM results

- Dataset is downloaded from [here](https://figshare.com/articles/dataset/BSD100_Set5_Set14_Urban100/21586188)

This asks for an alpha value to input and outputs processed images in a folder corresponding to its alpha value (e.g., alpha1.0, alpha2.2, etc.) named `outputs` in the same directory

In [32]:
import cv2
import numpy as np
import os
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
from skimage.metrics import peak_signal_noise_ratio as psnr_score
from skimage.metrics import structural_similarity as ssim_score
from glob import glob

"""For each pixel in the image, a small neighborhood around that pixel was considered using a 
blur-based approach to compute local mean and standard deviation. These local statistics guided 
the enhancement by adjusting the pixel intensity relative to the local mean, scaled by a factor 
that decreases in regions with high variation."""

# Parameters
window_size = 15 # 15x15 chosen based on the document (Methods section 3.1: Choice of Window Size)

bilateral_d = 9  # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
bilateral_sigmaColor = 75 # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
bilateral_sigmaSpace = 75 # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
alpha_global = float(input("enter alpha value: ")) # change alpha value here

def local_stats(image, ksize):
    img_f = image.astype(np.float32) / 255.0
    local_mean = cv2.blur(img_f, (ksize, ksize))
    sqr_mean = cv2.blur(img_f**2, (ksize, ksize))
    local_std = np.sqrt(sqr_mean - local_mean**2 + 1e-10)
    return local_mean, local_std

def adaptive_contrast_enhancement(img, ksize):
    local_mean, local_std = local_stats(img, ksize)
    max_std = local_std.max() if local_std.max() > 0 else 1
    norm_std = local_std / max_std
    alpha = alpha_global # ALPHA IS CHANGED HERE
    enhanced = local_mean + alpha * (img/255.0 - local_mean) * (1 - norm_std)
    enhanced = np.clip(enhanced, 0, 1)
    return (enhanced * 255).astype(np.uint8)

def process_image(path, output_dir):
    img = io.imread(path)
    original_gray = img_as_ubyte(rgb2gray(img)) if img.ndim == 3 else img

    enhanced = adaptive_contrast_enhancement(original_gray, window_size)
    filtered = cv2.bilateralFilter(enhanced, bilateral_d, bilateral_sigmaColor, bilateral_sigmaSpace)

    filename = os.path.basename(path)
    save_path = os.path.join(output_dir, filename)
    cv2.imwrite(save_path, filtered)

    # PSNR and SSIM vs original grayscale image
    psnr_val = psnr_score(original_gray, filtered, data_range=255)
    ssim_val = ssim_score(original_gray, filtered, data_range=255)

    print(f"{filename} - PSNR: {psnr_val:.2f} dB | SSIM: {ssim_val:.4f}")
    return psnr_val, ssim_val

def main():
    input_folder = 'BSD100\image_SRF_2'  # downloaded from https://figshare.com/articles/dataset/BSD100_Set5_Set14_Urban100/21586188
    output_folder = f'outputs\\alpha{alpha_global}'
    os.makedirs(output_folder, exist_ok=True)

    image_paths = glob(os.path.join(input_folder, '*.[jp][pn]g'))

    psnr_total, ssim_total = 0.0, 0.0
    count = 0

    for img_path in image_paths:
        psnr_val, ssim_val = process_image(img_path, output_folder)
        psnr_total += psnr_val
        ssim_total += ssim_val
        count += 1

    if count > 0:
        print(f"\nAverage PSNR: {psnr_total / count:.2f} dB")
        print(f"Average SSIM: {ssim_total / count:.4f}")

if __name__ == "__main__":
    main()


<>:59: SyntaxWarning: invalid escape sequence '\i'
<>:59: SyntaxWarning: invalid escape sequence '\i'
C:\Users\Jave\AppData\Local\Temp\ipykernel_24948\1362214137.py:59: SyntaxWarning: invalid escape sequence '\i'
  input_folder = 'BSD100\image_SRF_2'  # downloaded from https://figshare.com/articles/dataset/BSD100_Set5_Set14_Urban100/21586188


img_001_SRF_2_HR.png - PSNR: 25.71 dB | SSIM: 0.8387
img_001_SRF_2_LR.png - PSNR: 25.32 dB | SSIM: 0.8384
img_002_SRF_2_HR.png - PSNR: 28.33 dB | SSIM: 0.9046
img_002_SRF_2_LR.png - PSNR: 26.37 dB | SSIM: 0.8849
img_003_SRF_2_HR.png - PSNR: 27.01 dB | SSIM: 0.8761
img_003_SRF_2_LR.png - PSNR: 27.16 dB | SSIM: 0.8871
img_004_SRF_2_HR.png - PSNR: 28.40 dB | SSIM: 0.8791
img_004_SRF_2_LR.png - PSNR: 27.05 dB | SSIM: 0.8668
img_005_SRF_2_HR.png - PSNR: 27.23 dB | SSIM: 0.8682
img_005_SRF_2_LR.png - PSNR: 26.35 dB | SSIM: 0.8563
img_006_SRF_2_HR.png - PSNR: 29.44 dB | SSIM: 0.9113
img_006_SRF_2_LR.png - PSNR: 27.66 dB | SSIM: 0.8967
img_007_SRF_2_HR.png - PSNR: 25.81 dB | SSIM: 0.8572
img_007_SRF_2_LR.png - PSNR: 25.34 dB | SSIM: 0.8505
img_008_SRF_2_HR.png - PSNR: 25.31 dB | SSIM: 0.8428
img_008_SRF_2_LR.png - PSNR: 26.25 dB | SSIM: 0.8358
img_009_SRF_2_HR.png - PSNR: 26.01 dB | SSIM: 0.8670
img_009_SRF_2_LR.png - PSNR: 24.29 dB | SSIM: 0.8301
img_010_SRF_2_HR.png - PSNR: 28.71 dB | SSIM: 

# With GUI on a single selected photo

In [31]:
import cv2
import numpy as np
import os
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
from skimage.metrics import peak_signal_noise_ratio as psnr_score
from skimage.metrics import structural_similarity as ssim_score
from glob import glob
import tkinter as tk
from tkinter import filedialog

window_size = 15 # 15x15 chosen based on the document (Methods section 3.1: Choice of Window Size)

# Initial parameters for sliders
bilateral_d = 9 # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
bilateral_sigmaColor = 75 # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
bilateral_sigmaSpace = 75 # default values from https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
alpha_global = 15  # will represent 1.5 (divide by 10)

def local_stats(image, ksize):
    img_f = image.astype(np.float32) / 255.0
    local_mean = cv2.blur(img_f, (ksize, ksize))
    sqr_mean = cv2.blur(img_f**2, (ksize, ksize))
    local_std = np.sqrt(sqr_mean - local_mean**2 + 1e-10)
    return local_mean, local_std

def adaptive_contrast_enhancement(img, ksize, alpha):
    local_mean, local_std = local_stats(img, ksize)
    max_std = local_std.max() if local_std.max() > 0 else 1
    norm_std = local_std / max_std
    enhanced = local_mean + alpha * (img/255.0 - local_mean) * (1 - norm_std)
    enhanced = np.clip(enhanced, 0, 1)
    return (enhanced * 255).astype(np.uint8)

def process_image(img_gray, alpha, d, sigmaColor, sigmaSpace):
    enhanced = adaptive_contrast_enhancement(img_gray, window_size, alpha)
    filtered = cv2.bilateralFilter(enhanced, d, sigmaColor, sigmaSpace)
    return filtered

def nothing(x):
    pass

def load_image_from_path(path):
    img = io.imread(path)
    if img.ndim == 3:
        img_gray = img_as_ubyte(rgb2gray(img))
    else:
        img_gray = img
    return img_gray, os.path.basename(path)

def main():
    global bilateral_d, bilateral_sigmaColor, bilateral_sigmaSpace, alpha_global

    root = tk.Tk()
    root.geometry('800x600')   #  root window
    root.update()              # Update to apply size
    root.withdraw()            # Hide the root window

    img_path = filedialog.askopenfilename(
        title="Select an image",
        filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp;*.tiff")]
    )

    if not img_path:
        print("No image selected, exiting.")
        return

    img_gray, img_name = load_image_from_path(img_path)

    cv2.namedWindow('Contrast Enhancement')

    # Create trackbars
    cv2.namedWindow('Contrast Enhancement', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Contrast Enhancement', 1000, 600)
    cv2.createTrackbar('Alpha x0.1', 'Contrast Enhancement', alpha_global, 50, nothing)
    cv2.createTrackbar('d', 'Contrast Enhancement', bilateral_d, 20, nothing)
    cv2.createTrackbar('S Color', 'Contrast Enhancement', bilateral_sigmaColor, 150, nothing)
    cv2.createTrackbar('S Space', 'Contrast Enhancement', bilateral_sigmaSpace, 150, nothing)

    while True:
        alpha_global = cv2.getTrackbarPos('Alpha x0.1', 'Contrast Enhancement')
        bilateral_d = cv2.getTrackbarPos('d', 'Contrast Enhancement')
        bilateral_sigmaColor = cv2.getTrackbarPos('S Color', 'Contrast Enhancement')
        bilateral_sigmaSpace = cv2.getTrackbarPos('S Space', 'Contrast Enhancement')

        if bilateral_d <= 0:
            bilateral_d = 1
        if bilateral_d % 2 == 0:
            bilateral_d += 1
            if bilateral_d > 21:
                bilateral_d = 21

        alpha = max(0.1, min(alpha_global / 10, 5.0))

        processed = process_image(img_gray, alpha, bilateral_d, bilateral_sigmaColor, bilateral_sigmaSpace)

        psnr_val = psnr_score(img_gray, processed, data_range=255)
        ssim_val = ssim_score(img_gray, processed, data_range=255)

        original_bgr = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
        processed_bgr = cv2.cvtColor(processed, cv2.COLOR_GRAY2BGR)
        combined = np.hstack((original_bgr, processed_bgr))

        height, width = combined.shape[:2]
        black_strip = np.zeros((50, width, 3), dtype=np.uint8)

        cv2.putText(black_strip, f'Original: {img_name}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
        cv2.putText(black_strip, f'Processed PSNR: {psnr_val:.2f} dB', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
        cv2.putText(black_strip, f'Processed SSIM: {ssim_val:.4f}', (width//2 + 10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
        cv2.putText(black_strip, f'Alpha: {alpha:.2f}', (width//2 + 10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

        final_img = np.vstack((combined, black_strip))

        cv2.imshow('Contrast Enhancement', final_img)

        key = cv2.waitKey(50) & 0xFF
        if key == 27:
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 